In [43]:
import pandas as pd
import numpy as np

from scipy.stats import ttest_1samp, ttest_ind, ttest_rel, shapiro, wilcoxon, mannwhitneyu, f_oneway, pearsonr, chisquare, chi2_contingency, ranksums, levene, bartlett, kruskal

#### 제3유형 모집단 2개

검정방법

1. 대응표본(쌍체) : 동일한 객체의 전 vs 후 평균 비교
  - 정규성 O : 대응표본 t-test(paired t-test)  - ttest_rel 사용
  - 정규성 X : 윌콕슨 부호순위 검정 - wilcoxon
  
2. 독립표본 : A집단 평균 vs B집단 평균
  - 정규성 O : 독립표본 t-test - ttest_ind 사용
  - 정규성 X : 윌콕슨 순위합 검정 - ranksums 사용

대응표본(쌍체) t검정 (paired t-test)

가설검정 순서

1. 가설 설정
2. 유의수준 확인
3. 정규성 검정 (주의) 차이값에 대한 정규성!
4. 검정 실시 (통계량, p-value)
5. 귀무가설 기각 여부 결정

독립표본 t검정 (2sample t-test)

가설검정 순서

1. 가설 설정
2. 유의수준 확인
3. 정규성 검정  (주의) 두 집단 모두 정규성을 만족해야 모수 검정이 가능하다!
4. 등분산성 검정 
5. 검정 실시 (통계랑, p-value)  (주의) 등분산성 여부 확인 (equal_var)
6. 귀무가설 기각여부 결정

예상 문제

Case 1) 대응표본(쌍체) t검정(paired t-test)

문제 1-1
다음은 혈압약을 먹은 전, 후의 혈압 데이터이다.
혈압약을 먹기 전, 후의 차이가 있는지 쌍체 t 검정을 실시하시오

(유의수준 5%)

* before : 혈압약을 먹기 전 혈압, after : 혈압약을 먹은 후의 혈압

* 귀무가설(H0) : 먹기 전, 후가 같다  (after - before = 0)
* 대립가설(H1) : 먹기 전, 후가 다르다 (after - before <> 0)

In [44]:
# 데이터 만들기

df = pd.DataFrame( {
    'before': [120, 135, 122, 124, 135, 122, 145, 160, 155, 142, 144, 135, 167],
    'after' : [110, 132, 123, 119, 123, 115, 140, 162, 142, 138, 135, 142, 160] })

In [45]:
# 정규성 검정
diff = df['after'] - df['before']

n_statistic, n_pvalue = shapiro(diff)

print(round(n_statistic,4), round(n_pvalue,4))

if n_pvalue > 0.05 :
  print('정규성을 따른다')
else :
  print('정규성을 따르지 않는다')

0.9589 0.7363
정규성을 따른다


In [46]:
# t-test 실시

statistic, pvalue = ttest_rel(df['after'],df['before'],alternative='two-sided')    # 순서 주의!!! 부호가 바뀌기 때문에 확인 필요

print(round(statistic,4), round(pvalue,4))

if pvalue > 0.05 :
  print('혈압약을 복용한 전, 후가 같다')
else :
  print('혈압약을 복용한 전, 후가 다르다')

-3.1382 0.0086
혈압약을 복용한 전, 후가 다르다


In [47]:
# 정규성을 따르지 않는 경우

nx_statistic, nx_pvalue = wilcoxon(diff, alternative='two-sided')

print(round(nx_statistic,4), round(nx_pvalue,4))

11.0 0.0134


예상 문제

Case 2) 독립표본 t검정 (2sample t-test)

문제2-1
다음은 A그룹과 B그룹 인원의 혈압 데이터이다.

두 그룹의 혈압 평균이 다르다고 할 수 있는지 독립표본 t 검정을 실시하시오. (양측 검정)

유의수준 5%

- A : A그룹 인원의 혈압, B : B그룹 인원의 혈압
- H0(귀무가설) : A=B
- H1(대립가설) : A<>B

In [48]:
# 데이터 만들기

df = pd.DataFrame({
  'A' : [120, 135, 122, 124, 135, 122, 145, 160, 155, 142, 144, 135, 167],
  'B' : [110, 132, 123, 119, 123, 115, 140, 162, 142, 138, 135, 142, 160]
})

print(df.head())

     A    B
0  120  110
1  135  132
2  122  123
3  124  119
4  135  123


In [49]:
# A, B 각각 정규성 검정 (scipy.stats.shapiro)
# 둘다 정규성을 만족하여야 한다! 하나라도 만족하지 않을 경우 윌콕슨 순위합 비모수 검정 실시

a_statistic, a_pvalue = shapiro(df['A'])
b_statistic, b_pvalue = shapiro(df['B'])

print('A의 검정통계량 : ', round(a_statistic,4), 'A의 p-value : ', round(a_pvalue,4))
print('B의 검정통계량 : ', round(b_statistic,4), 'B의 p-value : ', round(b_pvalue,4))

if a_pvalue > 0.05 and b_pvalue > 0.05 :
  print("A, B 모두 정규성을 만족한다")
else :
  print("A, B 둘 중 하나 이상 정규성을 만족하지 않는다")

A의 검정통계량 :  0.9314 A의 p-value :  0.3559
B의 검정통계량 :  0.9498 B의 p-value :  0.5956
A, B 모두 정규성을 만족한다


In [50]:
# 독립표본 검정에서는 등분산성을 고려하여야 한다 (by levene, bartlett)
# 귀무가설(H0) : 등분산성을 가진다
# 대립가설(H1) : 등분산성을 가지지 않는다

eq_statistic1, eq_pvalue1 = levene(df['A'], df['B'])
eq_statistic2, eq_pvalue2 = bartlett(df['A'], df['B'])

print(round(eq_pvalue1,4), round(eq_pvalue2,4))

0.8862 0.8673


In [51]:
# 정규성과 등분산성을 모두 만족하는 경우 t-test 진행  (등분산을 하지 않으면 equal_var을 False로 지정한다)

# 정규성이 확실한 경우 : barlett
# 정규성이 확실하지 않은 경우 : levene

statistic, pvalue = ttest_ind(df['A'], df['B'], equal_var=True, alternative='two-sided')

print(round(statistic,4), round(pvalue,4))

if pvalue > 0.05 :
  print('두 그룹의 혈압 평균은 같다 (귀무가설 채택)')
else :
  print("두 그룹 간 혈압 평균이 다르다 (귀무가설 기각)")

0.8192 0.4207
두 그룹의 혈압 평균은 같다 (귀무가설 채택)


In [52]:
# 정규성을 만족하지 않는 경우 비모수 검정 (순위합 검정 실시)

statistic, pvalue = ranksums(df['A'], df['B'], alternative="two-sided")

print(round(pvalue,4))

0.3975


문제2-2
다음은 A그룹과 B그룹 인원의 혈압 데이터이다.

A그룹의 혈압 평균이 B그룹보다 크다고 할 수 있는지 독립표본 t검정을 실시하시오. (단측검정)

유의수준 5%

- 귀무가설(H0) : A <= B   (A그룹이 B그룹보다 작거나 같다)
- 대립가설(H1) : A > B    (A그룹이 B그룹보다 크다)

In [54]:
# 정규성 검정
# 각각 정규성을 검증하여 둘다 귀무가설 채택, 즉 정규성을 만족해야 t-test가 가능하다!

a_statistic, a_pvalue = shapiro(df['A'])
b_statistic, b_pvalue = shapiro(df['B'])

print(a_pvalue, b_pvalue)

0.35585272312164307 0.5955665707588196


In [55]:
# 등분산성 검정
# 둘다 정규성을 만족하는 경우 bartlett 사용

eq_statistic, eq_pvalue = bartlett(df['A'], df['B'])

print(eq_pvalue)

0.867271716287991


In [59]:
# 정규성과 등분산성을 모두 만족하는 경우 t-test 실시

statistic, pvalue = ttest_ind(df['A'], df['B'], equal_var=True, alternative='greater')

print(pvalue)
if pvalue > 0.05 :
  print('A그룹은 B그룹보다 작거나 같다 (귀무가설 채택)')
else :
  print('A그룹은 B그룹보다 크다')

0.2103743106970833
A그룹은 B그룹보다 작거나 같다 (귀무가설 채택)


In [62]:
# 만약 정규성을 만족하지 않는 경우 (윌콕슨 순위합 검정 실시)

x_statistic, x_pvalue = ranksums(df['A'], df['B'], alternative='greater')

print(x_pvalue)
if x_pvalue > 0.05 :
  print('A그룹은 B그룹보다 작거나 같다 (귀무가설 채택)')
else :
  print('A그룹은 B그룹보다 크다')

0.19873346271129638
A그룹은 B그룹보다 작거나 같다 (귀무가설 채택)
